### Explore driver data

In [ ]:
import xarray as xr
import os
import math
import pandas as pd
import glob
import numpy as np

In [ ]:
path_to_orignal_data = '/glade/scratch/adamhb/my_subset_data/CZ2_wrf_1950_1980/CLM1PT_data'
files = sorted(os.listdir(path_to_orignal_data))
output_path = '/glade/scratch/adamhb/my_subset_data/CZ2_wrf_pre_industrial_t_adjust/CLM1PT_data'

In [ ]:
# Molar mass of water vapour [kg/mol]
mm_h2o  = 0.01801505
# Molar mass of dry air [kg/mol]
mm_dry  = 0.02897
# Molar mass ratio
eps_mol = mm_h2o / mm_dry
# Saturation vapour pressure at 273.15K [Pa]
esat_0C = 611.65685464
frac_2_pc = 100
degC_2_K  = 273.15

In [ ]:
# Import the original data
full_paths = [os.path.join(path_to_orignal_data, fname) for fname in files[:24]]
d = xr.open_mfdataset(full_paths)

In [ ]:
# Temp adjustment
def adjust_t(original_t,bias):
    return original_t * bias

def put_adjusted_temp_in_array(xds,bias):
    TBOT_array = list(xds.TBOT.values)
    
    adjusted_temp_array = np.zeros(len(TBOT_array))
    for i in range(len(TBOT_array)):
        adjusted_temp_array[i] = adjust_t(TBOT_array[i],bias=bias)

    return adjusted_temp_array

def add_adjusted_temp_to_data(xds,bias):
    times = xds.time.values
    n = len(times)
    adjusted_t_array = put_adjusted_temp_in_array(xds,bias=bias)
    
    adjusted_t_xda = xr.DataArray(
                 data = np.array(adjusted_t_array).reshape(n,1,1),
                 dims = ['time','lat','lon'],
                 coords = {"time": times},
                 attrs = {
                     'longname': 'air temperature at 2m',
                     'units' : 'K',
                     'mode' : 'time-dependent'
                     })
    
    xds['TBOT'] = adjusted_t_xda

    
# Get RH
    
def getRH_from_QBOT(QBOT,PSRF,TBOT):
    
    #print("QBOT:",QBOT)
    #print("PSRF:",PSRF)
    #print("TBOT:",TBOT)
    
    
    EBOT = PSRF * QBOT / (eps_mol + (1. - eps_mol) * QBOT)
    ESAT = esat_0C * math.exp( 17.67 * (TBOT - degC_2_K) / (TBOT - 29.65))
    RH   = frac_2_pc * EBOT / ESAT
    #print("RH",RH)
    
    return RH

    
def put_RH_in_array(xds):
    QBOT_array = list(xds.QBOT.values)
    PSRF_array = list(xds.PSRF.values)
    TBOT_array = list(xds.TBOT.values)
    times = xds.time.values

    RH_array = np.zeros(len(QBOT_array))
    for i in range(len(QBOT_array)):
        RH_array[i] = getRH_from_QBOT(QBOT_array[i],PSRF_array[i],TBOT_array[i])

    return RH_array
  
def add_RH_to_data(xds):
    times = xds.time.values
    n = len(times)
    RH_array = put_RH_in_array(xds)

    RH_xda = xr.DataArray(
                 data = np.array(RH_array).reshape(n,1,1),
                 dims = ['time','lat','lon'],
                 coords = {"time": times},
                 attrs = {
                     'longname': 'relative humidity',
                     'units' : '%',
                     'mode' : 'time-dependent'
                     })
    xds['RH'] = RH_xda
    
    
def adjust_temp_and_RH(file,bias):
    print(file)
    ds = xr.open_dataset(file, decode_times = False)
    
    #get time attributes
    time_attrs = ds.time.attrs
    
    #add variables
    add_adjusted_temp_to_data(ds,bias=bias)
    add_RH_to_data(ds)
        
    #re-assign time attributes to data
    ds.time.attrs = time_attrs
    
    #write output
    new_file_name = output_path + "/" + os.path.basename(file)
    ds.to_netcdf(new_file_name, format = "NETCDF3_64BIT", mode = "w")
    ds.close()
    print("Finished",new_file_name)

In [ ]:
output_path

In [ ]:
files = glob.glob(path_to_orignal_data + "/*1951*.nc")
print(sorted(files))

In [ ]:
for f in sorted(files):
    adjust_temp_and_RH(f,bias = 0.985)

### Check adjusted temp compared to original

In [ ]:
files[:24]

In [ ]:
# Import the original data
full_paths = [os.path.join(path_to_orignal_data, fname) for fname in files[:24]]
print(full_paths)
d = xr.open_mfdataset(full_paths)

# Import the altered data
output_files = sorted(os.listdir(output_path))

full_paths = [os.path.join(output_path, fname) for fname in output_files[:24]]
print(full_paths)
d_adjusted = xr.open_mfdataset(full_paths)

In [ ]:
d.TBOT.sel(time = slice("1951-01-01","1951-02-02")).plot()
d_adjusted.TBOT.sel(time = slice("1951-01-01","1951-02-02")).plot()

In [ ]:
d.RH.sel(time = slice("1951-01-01","1951-02-02")).plot()
d_adjusted.RH.sel(time = slice("1951-01-01","1951-02-02")).plot()

In [ ]:
d.QBOT

In [ ]:
for v in ["TBOT","PSRF","QBOT","RH"]:
    print(v,d[v].isel(time = 100).values)


In [ ]:
def compute_relative_humidity(specific_humidity, pressure_pa, temperature_k):
    # Compute the saturation vapor pressure using the Tetens formula
    es = 0.611 * 10 ** (7.5 * (temperature_k - 273.15) / (237.3 + (temperature_k - 273.15)))
    es = es * 1000  # Convert kPa to Pa
    
    # Calculate relative humidity
    rh = (specific_humidity * pressure_pa) / (0.622 * es)
    return rh * 100  # Return as percentage

# Example usage:
specific_humidity = 0.00271898  # Example value in kg/kg
pressure_pa = 83116.03  # Example value in Pa for standard atmospheric pressure
temperature_k = 273.80212 * 0.985  # Example value in Kelvin (20 degrees Celsius)

rh = compute_relative_humidity(specific_humidity, pressure_pa, temperature_k)
print(f"Relative Humidity: {rh:.2f}%")

In [ ]:
d.PSRF.sel(time = slice("1951-01-01","1951-11-02")).plot()

In [ ]:
d

In [ ]:
def cast_all_vars_to_float32(xds):
    vars = list(xds.keys())
    for v in vars:
        float_32_var = xds[v].astype("float32")
        xds[v] = float_32_var
    return xds

def change_LATIXY_and_LONGXY(xds,lat,lon):
    LONGXY_attrs = xds.LONGXY.attrs
    LATIXY_attrs = xds.LATIXY.attrs
    new_LATIXY = xr.DataArray(data = np.array(lat,dtype = "float32").reshape(1,1),
                              dims = ['lat','lon'],
                              attrs = LATIXY_attrs)
    new_LONGXY = xr.DataArray(data = np.array(lon,dtype = "float32").reshape(1,1),
                              dims = ['lat','lon'],
                              attrs = LONGXY_attrs)
    
    xds['LATIXY'] = new_LATIXY
    xds['LONGXY'] = new_LONGXY
    return xds

def getRH_from_QBOT(QBOT,PSRF,TBOT):

    EBOT = PSRF * QBOT / (eps_mol + (1. - eps_mol) * QBOT)
    ESAT = esat_0C * math.exp( 17.67 * (TBOT - degC_2_K) / (TBOT - 29.65))
    RH   = frac_2_pc * EBOT / ESAT
    return RH

    
def put_RH_in_array(xds):
    QBOT_array = list(xds.QBOT.values)
    PSRF_array = list(xds.PSRF.values)
    TBOT_array = list(xds.TBOT.values)
    times = xds.time.values

    RH_array = np.zeros(len(QBOT_array))
    for i in range(len(QBOT_array)):
        RH_array[i] = getRH_from_QBOT(QBOT_array[i],PSRF_array[i],TBOT_array[i])

    return RH_array


def add_RH_to_data(xds):
    times = xds.time.values
    n = len(times)
    RH_array = put_RH_in_array(xds)

    RH_xda = xr.DataArray(
                 data = np.array(RH_array).reshape(n,1,1),
                 dims = ['time','lat','lon'],
                 coords = {"time": times},
                 attrs = {
                     'longname': 'relative humidity',
                     'units' : '%',
                     'mode' : 'time-dependent'
                     })
    xds['RH'] = RH_xda

def add_ZBOT_to_data(xds):
    times = xds.time.values
    n = len(xds.time.values)                    
    ZBOT_array = np.array([site_refhgt] * n).reshape(n,1,1) 
                        
    ZBOT_xda = xr.DataArray(
                 data = ZBOT_array.reshape(n,1,1),
                 dims = ['time','lat','lon'],
                 coords = {"time": times},
                 attrs = {
                     'longname': 'observation_height',
                     'units' : 'm',
                     'mode' : 'time-dependent'
                     })
    xds['ZBOT'] = ZBOT_xda

    
def add_EDGE_vars(xds):
    
    add = [0.5,-0.5,0.5,-0.5]
    var = ['LATIXY','LATIXY','LONGXY','LONGXY']
    for i,d in enumerate(['N','S','E','W']):
        edge_var_name = "EDGE" + d
        edge_value = xds[var[i]].values + add[i]
        edge_xar = xr.DataArray(
                     data = edge_value[0,:],
                     dims = ['scalar'],
                     attrs = {
                     'longname': 'edge of datm',
                     'units' : 'degrees',
                     'mode' : 'time-dependent'
                     })
        xds[edge_var_name] = edge_xar

            
def subset_single_file(file):
    ds = xr.open_dataset(file, decode_times = False)
    
    #subset
    point_data = ds.sel(lat = slice(wrf_lat_index,(wrf_lat_index+1)),
                        lon = slice(wrf_lon_index,(wrf_lon_index+1)))
    
    #get time attributes
    time_attrs = point_data.time.attrs
    
    #add variables
    add_ZBOT_to_data(point_data)
    add_RH_to_data(point_data)
        
    #reset lat and lon to be same as surf data
    point_data = change_LATIXY_and_LONGXY(cast_all_vars_to_float32(point_data),target_lat,target_lon)
    
    #add edge values
    add_EDGE_vars(point_data)
    
    #re-assign time attributes to data
    point_data.time.attrs = time_attrs
    
    #write output
    new_file_name = output_path + "/" + os.path.basename(file)
    point_data.to_netcdf(new_file_name, format = "NETCDF3_64BIT", mode = "w")
    ds.close()
    point_data.close()
    print("Finished",new_file_name)

In [ ]:
# View variables

list(d.keys())

In [ ]:
d

In [ ]:
CZ2_lat = 37.0311
CZ2_lon = 240.7434 - 360

### Plot annual precip totals

In [ ]:
hourly_precip = d.PRECTmms * 3600
annual_precip = hourly_precip.groupby(d.time.dt.year).sum()
annual_precip.plot(marker = "o")

### Plot mean monthly precipitation

In [ ]:
#hourly_precip.groupby(d.time.dt.year).sum().plot()

mean_precip_rate_per_month = d.PRECTmms.groupby(d.time.dt.month).mean() * 30.4 * 24 * 3600
mean_precip_rate_per_month.plot()

### Plot temp

In [ ]:
temp_CZ2 = d.TBOT - 273
temp_CZ2.groupby(d.time.dt.month).mean().plot(marker = "o")

### Temp by month

In [ ]:
temp_monthly = temp_CZ2.groupby(d.time.dt.month).mean()
temp_monthly.plot(marker = "o")